In [1]:
# Code for ETL operations on Country-GDP data

# Importing the required libraries

import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import numpy as np
from datetime import datetime

In [97]:
def log_progress(message):
    ''' This function logs the mentioned message of a given stage of the
    code execution to a log file. Function returns nothing'''
    time_format = '%Y-%h-%d:%H-%M-%S'
    current_time = datetime.now()
    timestamp = current_time.strfttime(time_format)
    with open('code_log.txt', 'a') as f:
        f.write(timestamp + ' : '+ str(message))
        
def extract(url, table_attribs):
    ''' This function aims to extract the required
    information from the website and save it to a data frame. The
    function returns the data frame for further processing. '''
    df = pd.DataFrame(columns=table_attribs)

    for col in table.find_all('tr'):
        if len(col.find_all('td')) == 0:
            continue
        bank_dict = {table_attribs[1]: col.find_all('td')[1].get_text().strip(),
                     table_attribs[2]: float(col.find_all('td')[2].get_text().strip())}
        bank_df = pd.DataFrame(bank_dict, index=[0])
        df = pd.concat([df,bank_df], ignore_index=True) 
    return df

def transform(df, csv_path):
    ''' This function accesses the CSV file for exchange rate
    information, and adds three columns to the data frame, each
    containing the transformed version of Market Cap column to
    respective currencies'''
    exchange_rate = pd.read_csv(csv_path)
    exchange_rate_dict = exchange_rate.set_index('Currency').to_dict()['Rate']
    currency = ['MC_EUR_Billion' , 'MC_GBP_Billion', 'MC_INR_Billion']

    for i,j in zip(MC,exchange_rate_dict.keys()):
        df[i] = np.round(df['MC_USD_Billion'] * exchange_rate_dict[j],2)

def load_to_csv(df, output_path):
    ''' This function saves the final data frame as a CSV file in
    the provided path. Function returns nothing.'''
    df.to_csv(output_path, index=False)

def load_to_db(df, sql_connection, table_name):
    ''' This function saves the final data frame to a database
    table with the provided name. Function returns nothing.'''
    df.to_sql(table_name, sql_connection,if_exist = 'replace', index= False)

def run_query(query_statement, sql_connection):
    ''' This function runs the query on the database table and
    prints the output on the terminal. Function returns nothing. '''
    pd.read_sql(query_statement, sql_connection)

In [2]:
url = 'https://web.archive.org/web/20230908092635 /https://en.wikipedia.org/wiki/List_of_largest_banks'

In [3]:
webpage = requests.get(url).text

In [4]:
soup = BeautifulSoup(webpage,'html.parser')

In [5]:
table = soup.find('table')

In [6]:
table.find_all('tr')[3].find_all('td')[0].get_text().strip()

'3'

In [ ]:
table.find_all('tr')[3].find_all('td')[1].get_text().strip()

In [ ]:
banks_dict = {'rank':[],'bank_name':[],'market':[]}

for col in table.find_all('tr'):
    if len(col.find_all('td')) == 0:
        continue
    for index,key in enumerate(banks_dict.keys()):
        banks_dict[key].append(col.find_all('td')[index].get_text().strip())

banks_df = pd.DataFrame(banks_dict)

In [18]:
table_attribs = ['rank','bank_name','market']
banks_df = pd.DataFrame(columns=table_attribs)

In [ ]:
for col in table.find_all('tr'):
    if len(col.find_all('td')) == 0:
        continue
    print(col.find_all('td')[1].get_text().strip())

In [80]:
table_attribs = ['rank','bank_name','market']
banks_df = pd.DataFrame(columns=table_attribs)

for col in table.find_all('tr'):
    if len(col.find_all('td')) == 0:
        continue
    bank_dict = {table_attribs[0]: col.find_all('td')[0].get_text().strip(),
                 table_attribs[1]: col.find_all('td')[1].get_text().strip(),
                 table_attribs[2]: float(col.find_all('td')[2].get_text().strip())}
    bank_df = pd.DataFrame(bank_dict, index=[0])
    banks_df = pd.concat([banks_df,bank_df], ignore_index=True)
banks_df    

,rank,bank_name,market
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


In [82]:
exchange_rate = pd.read_csv('exchange_rate.csv')
exchange_rate

,Currency,Rate
0,EUR,0.93
1,GBP,0.80
2,INR,82.95


In [81]:
exchange_rate = pd.read_csv('exchange_rate.csv')
exchange_rate_dict = exchange_rate.set_index('Currency').to_dict()['Rate']

In [83]:
banks_df .rename(columns={'market':'MC_USD_Billion'},inplace=True)

In [84]:
banks_df['MC_GBP_Billion'] = [np.round(x*exchange_rate_dict['GBP'],2) for x in banks_df['MC_USD_Billion']]
banks_df

,rank,bank_name,MC_USD_Billion,MC_GBP_Billion
0,1,JPMorgan Chase,432.92,346.34
1,2,Bank of America,231.52,185.22
2,3,Industrial and Commercial Bank of China,194.56,155.65
3,4,Agricultural Bank of China,160.68,128.54
4,5,HDFC Bank,157.91,126.33
5,6,Wells Fargo,155.87,124.70
6,7,HSBC Holdings PLC,148.90,119.12
7,8,Morgan Stanley,140.83,112.66
8,9,China Construction Bank,139.82,111.86
9,10,Bank of China,136.81,109.45


In [76]:
exchange_rate_dict.keys()

dict_keys(['EUR', 'GBP', 'INR'])

In [85]:
exchange_rate = pd.read_csv('exchange_rate.csv')
exchange_rate_dict = exchange_rate.set_index('Currency').to_dict()['Rate']
currency = ['MC_EUR_Billion' , 'MC_GBP_Billion', 'MC_INR_Billion']

for i,j in zip(currency, exchange_rate_dict.keys()):
    banks_df[i] = [np.round(x*exchange_rate_dict[j],2) for x in banks_df['MC_USD_Billion']]

In [86]:
banks_df

,rank,bank_name,MC_USD_Billion,MC_GBP_Billion,MC_EUR_Billion,MC_INR_Billion
0,1,JPMorgan Chase,432.92,346.34,402.62,35910.71
1,2,Bank of America,231.52,185.22,215.31,19204.58
2,3,Industrial and Commercial Bank of China,194.56,155.65,180.94,16138.75
3,4,Agricultural Bank of China,160.68,128.54,149.43,13328.41
4,5,HDFC Bank,157.91,126.33,146.86,13098.63
5,6,Wells Fargo,155.87,124.70,144.96,12929.42
6,7,HSBC Holdings PLC,148.90,119.12,138.48,12351.26
7,8,Morgan Stanley,140.83,112.66,130.97,11681.85
8,9,China Construction Bank,139.82,111.86,130.03,11598.07
9,10,Bank of China,136.81,109.45,127.23,11348.39


In [87]:
banks_df_copy = banks_df.copy()

In [89]:
banks_df_copy.drop(columns= ['MC_EUR_Billion' , 'MC_GBP_Billion', 'MC_INR_Billion'], inplace = True)

In [93]:
exchange_rate_dict.keys()

dict_keys(['EUR', 'GBP', 'INR'])

In [94]:
MC = ['MC_EUR_Billion' , 'MC_GBP_Billion', 'MC_INR_Billion']

for i,j in zip(MC,exchange_rate_dict.keys()):
    banks_df_copy[i] = np.round(banks_df_copy['MC_USD_Billion'] * exchange_rate_dict[j],2)

In [95]:
banks_df_copy

,rank,bank_name,MC_USD_Billion,MC_EUR_Billion,MC_GBP_Billion,MC_INR_Billion
0,1,JPMorgan Chase,432.92,402.62,346.34,35910.71
1,2,Bank of America,231.52,215.31,185.22,19204.58
2,3,Industrial and Commercial Bank of China,194.56,180.94,155.65,16138.75
3,4,Agricultural Bank of China,160.68,149.43,128.54,13328.41
4,5,HDFC Bank,157.91,146.86,126.33,13098.63
5,6,Wells Fargo,155.87,144.96,124.70,12929.42
6,7,HSBC Holdings PLC,148.90,138.48,119.12,12351.26
7,8,Morgan Stanley,140.83,130.97,112.66,11681.85
8,9,China Construction Bank,139.82,130.03,111.86,11598.07
9,10,Bank of China,136.81,127.23,109.45,11348.39
